LEEME antes de empezar: 

1. Apuntamos hacia el csv que queremos leer (siempre tiene que estar en la misma carpeta, en caso contrario, poner la ruta de donde se encuentra en vuestro PC)
2. De ahí ya está preparado para que nos saque el nombre de peli, el año y el ID de imdb 
3. Hace una concatenación de año y nombre para luego buscarlo en tomatoes. 
4. Empezamos con IMDB: hay Ids que no existen y primero los ignoraba, pero despues los he añadido a reusltados todo con NA para que la LEN de este resultado sea = a la LEN de tomates y podamos fusionar y hacer DF (luego lo depuramos en sql con not null)
5. Tomatitos = saca el valor de tomatometer y lo guarda en una lista 
6. Cuando tengo ambas listas las concateno, /cada tomatometer irá en lapeli que toca ya que hemos añadido tambien los NULL para asegurar el orden 
7. Hacemos el Data Frame con la columna de tomatometer añadida al final 
8. Guardamos CSV con el nombre que toque en cada caso 

¡ÁNIMO!

LO UNICO QUE HAY QUE CAMBIAR EN CADA CASO: 

- df_romance = el csv de la variable df_romance
- df_total.to_csv("romance_fase2_1990_2023.csv") el nombre del csv que guardo 


In [18]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.
import re
from pandas import DataFrame
# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import ast
import json
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.

from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.common.exceptions import NoSuchElementException

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


# OJO aqui tenemos que cambiar el CSV al cual apuntemos, ahora está el "romance"

In [19]:
#EJEMPLO:cojo mi csv de pelis de romance para testear la iteración por ID en imdb 
#CAMBIAME!! 
df_romance = pd.read_csv("romance_test_fase2.csv", index_col=0)
df_romance.head()

,genre,type,name,year,month,id
2045,Romance,Movie,New Strains,2023,1,tt26345526
2046,Romance,Movie,Someone You Loved,2023,2,tt26351536
2047,Romance,Movie,Zindagi Tere Naan,2023,3,tt26351098
2048,Romance,Movie,Milosc do kwadratu jeszcze raz,2023,2,tt26369131
2049,Romance,Short,Whistling Cloud: Spinnin,2023,1,tt26421477


In [20]:
lista_ids = df_romance["id"]
lista_ids.head()

2045    tt26345526
2046    tt26351536
2047    tt26351098
2048    tt26369131
2049    tt26421477
Name: id, dtype: object

In [21]:
lista_names_pelis = df_romance["name"].values
lista_años_pelis = df_romance["year"].values

In [22]:
lista_año_peli_zip = list(zip(lista_names_pelis, lista_años_pelis))
lista_año_peli_zip

[('New Strains', 2023),
 ('Someone You Loved', 2023),
 ('Zindagi Tere Naan', 2023),
 ('Milosc do kwadratu jeszcze raz', 2023),
 ('Whistling Cloud: Spinnin', 2023),
 ('Public Indecency', 2023),
 ('Coming of Age: Love', 2023),
 ('Svobodnyye otnosheniya', 2023),
 ('Czerwone niebo', 2023),
 ('Mr. Pawsitively Perfect', 2023),
 ('Fursat', 2023),
 ('Lyogkoye znakomstvo', 2023)]

# PARTE 1 -- IMDB

In [23]:
# ABRIMOS NAVEGADOR ------------------------------------------------------

driver = webdriver.Chrome()

#Busca la web que yo quiera, en este caso IMDB
driver.get("https://www.imdb.com/")
#print("0 - Abriendo la web")

# Maximizar la ventana del navegador para mostrar la página completa
driver.maximize_window()


#------------------------- COMENZAMOS LOOP ---------------------------------

lista_imdb = []
for id in tqdm(lista_ids):
    lista_consumible = []
    
    try:

        driver.find_element("css selector", "#suggestion-search").send_keys(id, Keys.ENTER)
        sleep(4)

        lista_consumible.append(id)
        #print(" 3- Añadiendo ID")

#PUNTUACION ----------------------------------------------------------------
        try: 
            puntuacion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-3a4309f8-0.fjtZsE.sc-dffc6c81-1.fJrHDo > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.gtEgaf > div.sc-bde20123-2.gYgHoj > span.sc-bde20123-1.iZlgcd").text
            #print("4 - Sacando puntuación imdb")
            lista_consumible.append(puntuacion)
            sleep(4)
        except: 
            lista_consumible.append("NA")
            

#DIRECCION ----------------------------------------------------------------
        try: 

            director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text
            lista_consumible.append(director)
            #print("4 - Sacando Director")
            sleep(4)
        except: 
            lista_consumible.append("NA")

#GUIONISTAS --------------------------------------------------------------
        
        try: 
            guionistas = []

            for guionista in range(1,10):
                try: 
                    nombres_guionistas = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li:nth-child({guionista}) > a").text
                    guionistas.append(nombres_guionistas)
                except:
                    break

            lista_consumible.append(guionistas)
        except: 
            lista_consumible.append("NA")
            #print("5 - Sacando Guionistas")

#ARGUMENTO PELICULA -----------------------------------------------------
        try:

            
            argumento = driver.find_element("xpath", "/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]").text
            
        except NoSuchElementException:
            # Handle the case where the element is not found
            argumento = "Element not found"

        lista_consumible.append(argumento)

        sleep(4)

        #print("9 - Sacando Argumento")

#DURACION ---------------------------------------------------------------
        try: 
            duracion_horas = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(2)").text 
            #print("6 - Sacando duración")
            #PARA CONVERTIR LA DURACION TOTAL A MINUTOS! 
            partes = duracion_horas.split() 

            horas = 0 
            minutos = 0 

            for parte in partes:
                if "h" in parte: 
                    horas = horas + int(parte[:-1]) 
                elif "m" in parte: 
                    minutos = minutos + int(parte[:-1])# :-1 al eliminar el H y el M entonces nos da el digit. Si pongo [0]me daría solo el 2 de 22, o el 2 de 25.

            duracion = horas * 60 + minutos 

            lista_consumible.append(duracion)
            sleep(6)
            #print("7 - convirtiendo a minutos")
        except: 
            lista_consumible.append("NA")

#NOMBRE PELICULA ----------------------------------------------------------------
        try: 

            nombre_imdb = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > h1 > span").text
            sleep(4)
            lista_consumible.append(nombre_imdb)
            
            #print("8 - Sacando nombre Peli")
            sleep(4)
        except:
            lista_consumible.append("NA")

#----------------------------------------------------------------
# He añadido este parche para que me añada los casos de Ids de pelis que ya no existen en imdb pero que los busco igualmente en rotten tomatoes. 
# Luego se puede limpiar en BBDD pero así la len de ambas listas es = para hacer el dataframe y guardar el CSV 

    except NoSuchElementException:
        # Cuando el ID que estamos buscando no existe en imdb
        print(f"ID {id} does not exist on IMDb. Skipping...")
        puntuacion = "NA"
        lista_consumible.append(puntuacion)
        nombre_imdb = "NA"
        lista_consumible.append(nombre_imdb)
        duracion = "NA"
        lista_consumible.append(duracion)
        director = "NA"
        lista_consumible.append(director)
        argumento = "NA"
        lista_consumible.append(argumento)
        guionistas = "NA"
        lista_consumible.append(guionistas)

        driver.back()
        sleep(8)



# FUERA DEL LOOP: 

    if lista_consumible: #evita hacer append de tuplas vacías
                            
        lista_imdb.append(lista_consumible)
        #FALTA CONVERTIRLO A TUPLA


 25%|██▌       | 3/12 [01:20<03:36, 24.06s/it]

ID tt26369131 does not exist on IMDb. Skipping...


100%|██████████| 12/12 [06:17<00:00, 31.50s/it]


# FASE 2 - TOMATITOS 

In [26]:

# Website 2 - Rotten Tomatoes

# Inicializa el navegador de Chrome--------------------------
driver = webdriver.Chrome()

# Abre Google en el navegador--------------------------
driver.get("https://www.google.com/")
#print("1 - Abriendo Google")

# Maximiza la ventana del navegador--------------------------
driver.maximize_window()

# Acepta las cookies de Google--------------------------

try:
    driver.find_element(By.CSS_SELECTOR, "#L2AGLb > div").click()
    #print("2 - Cookies de Google")
except NoSuchElementException:
    print("El botón de cookies no se encontró, continuando con el código.")


lista_tomates = []

# Bucle para buscar las películas --------------------------
for año, título in lista_año_peli_zip:

# Realiza una búsqueda en Google --------------------------
    driver.get("https://www.google.com/")
    #print("3 - Entrando a Google"

    busqueda_pelicula = f"{año} {título} Rotten Tomatoes"
    # driver.find_element("css selector", "#APjFqb").clear()
    driver.find_element(By.CSS_SELECTOR, "#APjFqb").send_keys(
        busqueda_pelicula, Keys.ENTER
    )

    try:
        element = driver.find_element(
            By.XPATH,
            "/html/body/div[5]/div/div[12]/div/div[1]/div[1]/div/div/div[3]/div/div/div[2]/div/g-raised-button[1]/div/span",
        )

    except NoSuchElementException:
        print("No se encontró el elemento de sign up de google.")
        # SI ENTRO AQUI, IRÉ DIRECTAMENTE EL SIGUIENTE TRY (ES COMO UN PASS)
    else:
        element.click()

# Haz clic en el primer resultado de búsqueda ----------------------
    try:
        element = driver.find_element(By.XPATH, "//*[@id='rso']/div[1]/descendant::h3")

    except NoSuchElementException:
        print(
            "No se encontró el elemento. Puede ser necesario actualizar el selector CSS."
        )

        continue

    element.click()

    #print("5 - Haciendo clic en la búsqueda")

# Acepta las cookies en la página de Rotten Tomatoes --------------------------

    try:
        element = driver.find_element(By.CSS_SELECTOR, "#onetrust-accept-btn-handler")

    except NoSuchElementException:
        print("No se encontró las cookies de rotten")
    # SI ENTRO AQUI, IRÉ DIRECTAMENTE EL SIGUIENTE TRY (ES COMO UN PASS)
    else:
        element.click()

    #print("6 - Aceptando cookies en Rotten Tomatoes")

    try:
# Accede a la URL de Rotten Tomatoes para obtener información --------------------------

        tomatometer_element = driver.find_element("xpath", "//*[@id='scoreboard']")
        tomatometer = tomatometer_element.get_attribute("tomatometerscore")

        if tomatometer:
            lista_tomates.append([f"{tomatometer}%"])
            #print(f"Tomatometer Value for {año} - {título}: {tomatometer}")
        else:
            lista_tomates.append(["--"])
 
            #print("Tomatometer not found")
    except NoSuchElementException:
        lista_tomates.append(["--"])
    
        #print("No se encontró el tomato")

driver.quit()



No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten
No se encontró el elemento de sign up de google.
No se encontró las cookies de rotten


In [ ]:
df_imdb = pd.DataFrame(lista_imdb, columns=["ID", "Puntuación", "Dirección", "Guionistas", "Argumento", "Duración", "Nombre imdb"])
df_imdb

,ID,Puntuación,Dirección,Guionistas,Argumento,Duración,Nombre imdb
0,tt26345526,7.5,Prashanth Kamalakanthan,"[Prashanth Kamalakanthan, Artemis Shaw]","As a mysterious pandemic emerges, a squabbling...",78,New Strains
1,tt26351536,6.4,Seul-woo Hyung,[Seul-woo Hyung],Joon Ho and Ah Yeong have been best friends si...,103,Someone You Loved
2,tt26351098,NA,NA,[],Element not found,NA,NA
3,NA,NA,NA,NA,NA,NA,None
4,tt26421477,NA,Mun Chee Yong,[Mun Chee Yong],Element not found,NA,Whistling Cloud: Spinnin
5,tt26423913,7.1,Khaled Khella,[Khaled Khella],A conflicted young man and his overexcited gir...,30,Public Indecency
6,tt26425315,NA,Vaughn Arterberry,[Vaughn Arterberry],A man and woman discover a deeper connection b...,5,Coming of Age: Love
7,tt26438468,5.8,Dmitry Nevzorov,"[Nikita Lundin, Dmitry Nevzorov]",The story of two opposites who accidentally me...,102,Svobodnyye otnosheniya
8,tt26440619,7.3,Christian Petzold,[Christian Petzold],A group of friends are in a holiday home by th...,0,Afire
9,tt26446577,6.8,Graeme Campbell,[Anne Hollister],An aspiring marketing designer's favorite fost...,NA,Mr. Pawsitively Perfect


In [ ]:
len(lista_imdb)

12

In [ ]:
len(lista_tomates)

12

In [ ]:
df_tomates = pd.DataFrame(lista_tomates, columns=["tomatometer"])
df_tomates

,tomatometer
0,sin tomate
1,sin tomate
2,not found
3,17%
4,not found
5,74%
6,27%
7,81%
8,85%
9,sin tomate


# FUSIONAMOS AMBAS LISTAS DE RESULTADOS 

In [35]:
lista_imdb 
lista_tomates 
listas_imdb_tomate = []

for imdb, tomates in zip(lista_imdb, lista_tomates):
    lista_combinada = imdb + tomates
    listas_imdb_tomate.append(tuple(lista_combinada))

print(listas_imdb_tomate)

[('tt26345526', '7.5', 'Prashanth Kamalakanthan', ['Prashanth Kamalakanthan', 'Artemis Shaw'], 'As a mysterious pandemic emerges, a squabbling couple in lockdown begin to act like children.', 78, 'New Strains', '--'), ('tt26351536', '6.4', 'Seul-woo Hyung', ['Seul-woo Hyung'], 'Joon Ho and Ah Yeong have been best friends since college and now a couple living together. Ah Yeong is making the best of her situation, but she feels his boyfriend Joon Ho is wasting his life in excuse of preparing for an exam.', 103, 'Someone You Loved', '--'), ('tt26351098', 'NA', 'NA', [], 'Element not found', 'NA', 'NA', '--'), ('NA', 'NA', 'NA', 'NA', 'NA', 'NA', '17%'), ('tt26421477', 'NA', 'Mun Chee Yong', ['Mun Chee Yong'], 'Element not found', 'NA', 'Whistling Cloud: Spinnin', '--'), ('tt26423913', '7.1', 'Khaled Khella', ['Khaled Khella'], 'A conflicted young man and his overexcited girlfriend wander around the city of Cairo looking for a romantic spot to kiss. They unwittingly set off a squad to hun

In [36]:

listas_imdb_tomate

[('tt26345526',
  '7.5',
  'Prashanth Kamalakanthan',
  ['Prashanth Kamalakanthan', 'Artemis Shaw'],
  'As a mysterious pandemic emerges, a squabbling couple in lockdown begin to act like children.',
  78,
  'New Strains',
  '--'),
 ('tt26351536',
  '6.4',
  'Seul-woo Hyung',
  ['Seul-woo Hyung'],
  'Joon Ho and Ah Yeong have been best friends since college and now a couple living together. Ah Yeong is making the best of her situation, but she feels his boyfriend Joon Ho is wasting his life in excuse of preparing for an exam.',
  103,
  'Someone You Loved',
  '--'),
 ('tt26351098', 'NA', 'NA', [], 'Element not found', 'NA', 'NA', '--'),
 ('NA', 'NA', 'NA', 'NA', 'NA', 'NA', '17%'),
 ('tt26421477',
  'NA',
  'Mun Chee Yong',
  ['Mun Chee Yong'],
  'Element not found',
  'NA',
  'Whistling Cloud: Spinnin',
  '--'),
 ('tt26423913',
  '7.1',
  'Khaled Khella',
  ['Khaled Khella'],
  'A conflicted young man and his overexcited girlfriend wander around the city of Cairo looking for a romanti

# YA TENEMOS TOMATOMETER AÑADIDO 

In [37]:
df_total = pd.DataFrame(listas_imdb_tomate, columns=["ID", "Puntuación", "Dirección", "Guionistas", "Argumento", "Duración", "Nombre imdb", "tomatometer"])
df_total

,ID,Puntuación,Dirección,Guionistas,Argumento,Duración,Nombre imdb,tomatometer
0,tt26345526,7.5,Prashanth Kamalakanthan,"[Prashanth Kamalakanthan, Artemis Shaw]","As a mysterious pandemic emerges, a squabbling...",78,New Strains,--
1,tt26351536,6.4,Seul-woo Hyung,[Seul-woo Hyung],Joon Ho and Ah Yeong have been best friends si...,103,Someone You Loved,--
2,tt26351098,NA,NA,[],Element not found,NA,NA,--
3,NA,NA,NA,NA,NA,NA,17%,None
4,tt26421477,NA,Mun Chee Yong,[Mun Chee Yong],Element not found,NA,Whistling Cloud: Spinnin,--
5,tt26423913,7.1,Khaled Khella,[Khaled Khella],A conflicted young man and his overexcited gir...,30,Public Indecency,74%
6,tt26425315,NA,Vaughn Arterberry,[Vaughn Arterberry],A man and woman discover a deeper connection b...,5,Coming of Age: Love,27%
7,tt26438468,5.8,Dmitry Nevzorov,"[Nikita Lundin, Dmitry Nevzorov]",The story of two opposites who accidentally me...,102,Svobodnyye otnosheniya,81%
8,tt26440619,7.3,Christian Petzold,[Christian Petzold],A group of friends are in a holiday home by th...,0,Afire,85%
9,tt26446577,6.8,Graeme Campbell,[Anne Hollister],An aspiring marketing designer's favorite fost...,NA,Mr. Pawsitively Perfect,--


# Guardar todo lo que hagamos con nombre de genero + fase + año inicio + año fin

#vamos cambiando el nombre del csv por cada tramo de pelis que hagamos

In [ ]:
df_total.to_csv("romance_fase2_ejemplo.csv")